# Try AutoMLJar (Notebook Window Should be Opened)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1.0 Importing Essential Libraries

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf

# 2.0 Data Exploration & Pre-processing

In [ ]:
path = '../input/tabular-playground-series-jan-2021/'

train_df = pd.read_csv(path + 'train.csv')
test_df = pd.read_csv(path + 'test.csv')
sample_submission = pd.read_csv(path + 'sample_submission.csv')

In [ ]:
train_df.shape, test_df.shape, sample_submission.shape

In [ ]:
train_df.dtypes

In [ ]:
train_df.describe()

In [ ]:
train_df.head()

In [ ]:
X_train = train_df.drop(columns = ['target'])
#X_test = test_df.drop('id', axis = 1)
#y_train = cudf.DataFrame(train_df.target)
y_train = train_df['target']

In [ ]:
#X_train.set_index('id', inplace = True)
#X_train.head()

In [ ]:
X_test = test_df.copy()
#X_test.set_index('id', inplace = True)
#X_test.head()

In [ ]:
#columns = [col for col in train_df.columns.to_list() if col not in ['id','target']]
#X_train = train_df[columns]
#y_train = train_df['target']

In [ ]:
X_train.shape, y_train.shape, X_test.shape

# 3.0 Model Building in Pipelines

In [ ]:
#params1 = {
#    'n_estimators' : [100, 200, 300, 400, 500, 600],
#    'max_depth' : 10, #[4, 6, 8, 10, 12, 14],
#    'max_leaves' : 2**4,
#    'tree_method' : 'gpu_hist',
#    'learning_rate' : 0.1,
#    'objective' : 'reg:squaredlogerror',
#    'eval_metric' : 'rmse'
#}

In [ ]:
#from cuml.preprocessing.model_selection import train_test_split
from xgboost import XGBRegressor
#from cuml.metrics import roc_auc_score, mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
def objective(trial, X = X_train, y = y_train):
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, 
                                test_size = 0.2, random_state = 0)
    params = {
        'tree_method' : 'gpu_hist',
        'learning_rate' : trial.suggest_categorical("learning_rate",
            [0.008, 0.009, 0.01, 0.012, 0.014, 0.016 ,0.018, 0.02]),
        'n_estimators' : trial.suggest_int("n_estimators", 100, 1500),
        'max_depth' : trial.suggest_int("max_depth", 2, 20),
        #'max_features' : trial.suggest_uniform("max_features", 0.01, 1.0),
        'random_state' : 0,
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300)
    }
    
    xgb = XGBRegressor(**params)
    xgb.fit(X_train, y_train, eval_set = [(X_val, y_val)], verbose = False)
    
    pred = xgb.predict(X_val)
    
    rmse = mean_squared_error(y_val, pred, squared = False)
    
    return rmse

In [ ]:
import optuna
study = optuna.create_study(direction = 'minimize')
study.optimize(objective, n_trials = 100)
print ('Total Finished Trials:', len(study.trials))
print('Best Trial:', study.best_trial.params)

In [ ]:
#Total Finished Trials: 100
#Best Trial1: {'learning_rate': 0.01, 'n_estimators': 1258, 'max_depth': 16, 'max_features': 0.4314231900564939, 'lambda': 0.06175177599003586, 
#                'alpha': 0.0301162104635596, 'colsample_bytree': 0.6, 'subsample': 0.8, 'min_child_weight': 282}


# Best Trial No 72: {'learning_rate': 0.012, 'n_estimators': 1457, 'max_depth': 13, 'lambda': 3.470813989036924, 'alpha': 0.04682398849757302, 
#                'colsample_bytree': 0.7, 'subsample': 0.5, 'min_child_weight': 227}


# Best Trail No 54 : {'learning_rate': 0.012, 'n_estimators': 1471, 'max_depth': 13, 'lambda': 5.054720822936409, 'alpha': 0.06441154220145467, 
#                      'colsample_bytree': 0.7, 'subsample': 0.5, 'min_child_weight': 237}

In [ ]:
Best_trial = {'tree_method' : 'gpu_hist', 'eval_metric' : 'rmse', 'learning_rate': 0.01, 'n_estimators': 1258, 'max_depth': 16, 'lambda': 0.06175177599003586, 
                'alpha': 0.0301162104635596, 'colsample_bytree': 0.6, 'subsample': 0.8, 'min_child_weight': 282, 
              'importance_type' : 'total_gain', 'objective' : 'reg:tweedie', 'grow_policy' : 'depthwise', 'random_state' : 0} #'max_features': 0.4314231900564939,

from xgboost import XGBRegressor


xgb = XGBRegressor(**Best_trial) #, n_estimators = 500, max_depth = 10, eval_metric = 'rmse',
xgb.fit(X_train, y_train)

In [ ]:
#1st Try

#{'tree_method': 'gpu_hist',
# 'n_estimators': 300,
# 'min_child_weight': 5,
# 'max_depth': 6,
# 'learning_rate': 0.1,
# 'gamma': 0.4}

# Score = 0.70158


#2nd Try

#{'tree_method': 'gpu_hist',
# 'n_estimators': 400,
# 'min_child_weight': 3.5,
# 'max_depth': 6,
# 'learning_rate': 0.15,
# 'gamma': 0.6}

# Score = 0.70225


#3rd Try

#Best_trial = {'tree_method' : 'gpu_hist', 'learning_rate': 0.01, 'n_estimators': 1258, 'max_depth': 16, 
#'max_features': 0.4314231900564939, 'lambda': 0.06175177599003586, 'alpha': 0.0301162104635596, 'colsample_bytree': 0.6, 
#                'subsample': 0.8, 'min_child_weight': 282, 'random_state' : 0}

In [ ]:
# Predicting the model
y_pred = xgb.predict(X_test)

In [ ]:
#from sklearn.metrics import roc_auc_score, mean_squared_error, mean_absolute_error, r2_score


#print('R2 Score : ', r2_score(y_val, y_pred_val))
#print('ROC AUC SCORE : ', roc_auc_score(y_val, y_pred_val))
#print('Mean Squared Error : ', mean_squared_error(y_val, y_pred_val))
#print('Root Mean Squared Error : ', (mean_squared_error(y_val, y_pred_val, squared = False)))

In [ ]:
X_train.shape, y_train.shape, X_test.shape

In [ ]:
# Creating a Submission File
sample_submission['target'] = y_pred
sample_submission.to_csv('submission26.csv', index = False)

In [ ]:
# Reading the submission file
submission = pd.read_csv('submission26.csv')
submission.shape